In [1]:
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import scipy.io as sciio
from time import time
import sklearn
# %matplotlib inline
from sklearn.datasets import make_blobs
from sklearn.decomposition import NMF
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import skimage
from sklearn import svm, metrics, datasets
from sklearn.svm import SVC
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearnex import patch_sklearn
from skimage.io import imread
from skimage.transform import resize
from skimage.transform import resize as imresize

from load_data import LOAD_IMAGE

patch_sklearn()

REAL_IMAG = True
NO_NMF = False
is_NMF = False


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


# 数据加载部分

In [2]:
# load data

# ----KC501-----
# image_dataset = load_image_files('/home/kc501/LJY/Alexnet/dataRCS')

# ----ICraft----
t_load_data = time()
image_dataset = LOAD_IMAGE(REAL_IMAG)
# image_dataset1 = load_image_files('.\dataRCS')
# image_dataset2 = load_image_files('.\dataRCS2')
# # flat_data = np.concatenate([image_dataset1.data, image_dataset2.data])
# # target = np.concatenate([image_dataset1.target, image_dataset2.target])
# # categories = np.concatenate([image_dataset1.categories, image_dataset2.categories])
# # descr = image_dataset1.descr
# categories = image_dataset1.target_names
# categories.extend(image_dataset2.target_names)
# print("fucked")
# image_dataset = Bunch(data=np.concatenate([image_dataset1.data, image_dataset2.data]),
#                       target=np.concatenate([image_dataset1.target, image_dataset2.target]),
#                       target_names=categories,
#                       DESCR=image_dataset1.DESCR)
print("Data load done in %0.3fs" % (time() - t_load_data))
# image_dataset_test = load_image_files("E:/RL_code/alex-net-image-classification-master/class3/val")

0 1
1 2
2 3
3 4
4 5
0 10
1 6
2 7
3 8
4 9
fucked
Data load done in 244.746s


# 数据集划分部分

In [3]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.2, random_state=101)

# NMF降维模块

In [4]:
n_components = 10
tolerance = 1e-2
max_iteration = 10000
if not NO_NMF:
    t_NMF = time()
    if is_NMF:
        # nmf = NMF(n_components=n_components, init='random', tol=tolerance, max_iter=max_iteration, verbose=True).fit(X_train)
        nmf = NMF(n_components=n_components, init='random', tol=tolerance, max_iter=max_iteration, verbose=True).fit(
            image_dataset.data)
        print("NMF done in %0.3fs" % (time() - t_NMF))
    else:
        # nmf=PCA(n_components).fit(X_train)
        nmf = PCA(n_components).fit(image_dataset.data)
        print("PCA done in %0.3fs" % (time() - t_NMF))
    X_train_nmf = nmf.transform(X_train)
    X_test_nmf = nmf.transform(X_test)

# SVM训练模块


In [6]:
f = open('./TZB.txt', 'a')
t_train = time()
kernel = 'rbf'
clf = SVC(C=10000, kernel=kernel, gamma="scale", class_weight='balanced', decision_function_shape='ovo', verbose=True)

if NO_NMF:
    clf = clf.fit(X_train, y_train)
    print("SVM done in %0.3fs" % (time() - t_train))
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
else:
    clf = clf.fit(X_train_nmf, y_train)
    print("SVM done in %0.3fs" % (time() - t_train))
    y_pred_train = clf.predict(X_train_nmf)
    y_pred_test = clf.predict(X_test_nmf)

y_true_train = y_train
y_true_test = y_test

f.write("n_components: {}\n".format(n_components))
f.write("kernel: {}\n".format(kernel))
f.write("Decomposition: {}\n".format("NMF" if is_NMF else "PCA"))
f.write("tolerance: {}\n".format(tolerance))
f.write("max_iteration: {}\n".format(max_iteration))
f.write("Acc on train data: {}\n".format(accuracy_score(y_true_train, y_pred_train)))
f.write("Acc on test data: {}\n".format(accuracy_score(y_true_test, y_pred_test)))
f.write("------------------------------------------------------------------------------\n")
f.close()
print("n_components: ", n_components)
print("Acc on train data:" + str(accuracy_score(y_true_train, y_pred_train)))

print("Acc on test data:" + str(accuracy_score(y_true_test, y_pred_test)))

[LibSVM]SVM done in 1.349s
n_components:  10
Acc on train data:0.8215
Acc on test data:0.242
